# Model Development

This example shows how to build, train, evaluate and deploy a model running on FPGA. Only Windows is supported. We use TensorFlow and Keras to build our model. We are going to use transfer learning, with ResNet50 as a featurizer. We don't use the last layer of ResNet50 in this case and instead add and train our own classification layer.

We will use the Kaggle Cats and Dogs dataset to train the classifier. The data set can be downloaded [here](https://www.microsoft.com/en-us/download/details.aspx?id=54765). Download the zip and extract to a directory named 'catsanddogs' under your user directory ("~/catsanddogs").

Please set up your environment as described in the [quick start](00_QuickStart.ipynb).

In [2]:
import os
import sys
import tensorflow as tf
import numpy as np
import amlrealtimeai
from amlrealtimeai import resnet50
import sys
!{sys.executable} -m pip install h5py
import h5py

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
mkl-random 1.0.1 requires cython, which is not installed.
mkl-fft 1.0.4 requires cython, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Model Construction
Load the files we are going to use for training and testing. By default this notebook uses only a very small subset of the Cats and Dogs dataset. That makes it run quickly, but doesn't create a very accurate classifier. You can improve the classifier by using more of the dataset.

We need to preprocess the input file to get it into the form expected by ResNet50. We've provided a default implementation of the preprocessing that you can use.

In [38]:
# Input images as a two-dimensional tensor containing an arbitrary number of images represented a strings
import amlrealtimeai.resnet50.utils
#datadir = os.path.expanduser("/Users/mialiu/Downloads/")
in_images = tf.placeholder(tf.string)
image_tensors = resnet50.utils.preprocess_array(in_images)
image_tensors = tf.placeholder(tf.int32,shape=(None,224, 224, 3))
#print(image_tensors.shape)
print(type(image_tensors))

<class 'tensorflow.python.framework.ops.Tensor'>


Alternatively, if you would like to customize the preprocessing, you can write your own preprocessor using TensorFlow operations.

The input to the classifier we are training is the set of features produced by ResNet50. To train the classifier we need to 
featurize the images using resnet50. We do this using a featurizer running on FPGA. You can also run the featurizer locally on CPU or GPU.

Go to our [GitHub repo](https://aka.ms/aml-real-time-ai) "docs" folder to learn how to create a Model Management Account and find the required information below.

In [39]:
subscription_id = "80defacd-509e-410c-9812-6e52ed6a0016"
resource_group = "CMS_FPGA_Resources"
model_management_account = "CMS_FPGA_1"

from amlrealtimeai.resnet50.model import RemoteQuantizedResNet50
model_path = os.path.expanduser('~/models')
featurizer = RemoteQuantizedResNet50(subscription_id, resource_group, model_management_account, model_path)
print(featurizer.version)

1.1.6-rc


Calling import_graph_def on the featurizer will create a service that runs the featurizer on FPGA.

In [40]:
featurizer.import_graph_def(include_top=False, input_tensor=image_tensors)
features = featurizer.featurizer_output

Registering model resnet50-1.1.6-rc-model
Successfully registered model resnet50-1.1.6-rc-model
Creating service featurizer-service-08cdb9
. . . . . . . . . 
Successfully created service featurizer-service-08cdb9


## Pre-compute features
Load the data set and compute the features. These can be precomputed because they don't change during training.

In [41]:
from tqdm import tqdm

def chunks(l, n):
    """Yield successive n-sized chunks from a file."""
    for i in range(0, l.shape[0], n):
    #for i in range(0, 50, n):
        yield l[i:i + n]

def read_files(files):
    contents = []
    for path in files:
        with open(path, 'rb') as f:
            contents.append(f.read())
    return contents
        
feature_list = []
f = h5py.File("/Users/mialiu/Downloads/jetImage_1k.h5",'r')
images = f['jetImagePt']
with tf.Session() as sess:
    for chunk in tqdm(chunks(images,10)):
        onechan = chunk
        image = np.stack([onechan, onechan, onechan],axis=-1)# 
        result= sess.run([features],feed_dict={image_tensors:image}) # 
        feature_list.extend(result[0])
feature_results = np.array(feature_list)
print(feature_results.shape)

100it [06:59,  4.20s/it]

(999, 1, 1, 2048)


In [30]:
f['jetImageE']
f['jets']

<HDF5 dataset "jets": shape (1000, 60), type "<f8">

Remove remote service

In [42]:
featurizer.cleanup_remote_service()

Deleting service 8d85645de0ca4618b228a3733d9cf23c
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Deleted service 8d85645de0ca4618b228a3733d9cf23c
Deleting model cdcb37b6ee0a404ea4dcf98147a6e9f7
Deleted model cdcb37b6ee0a404ea4dcf98147a6e9f7


## Add and Train the classifier
We use Keras to define and train a simple classifier.

In [43]:
from keras.models import Sequential
from keras.layers import Dropout, Dense, Flatten
from keras import optimizers

FC_SIZE = 1024
NUM_CLASSES = 5

model = Sequential()
model.add(Dropout(0.2, input_shape=(1, 1, 2048,)))
model.add(Dense(FC_SIZE, activation='relu', input_dim=(1, 1, 2048,)))
model.add(Flatten())
model.add(Dense(NUM_CLASSES, activation='sigmoid', input_dim=FC_SIZE))

model.compile(optimizer=optimizers.SGD(lr=1e-4,momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])

Prepare the train and test data.

In [45]:
from sklearn.model_selection import train_test_split
#onehot_labels = np.array([[0,1] if i else [1,0] for i in labels])
onehot_labels = np.array([f["jets"][i][-6:-1] for i in range(0,f["jets"].shape[0]-1)])
print(onehot_labels)
X_train, X_test, y_train, y_test = train_test_split(feature_results, onehot_labels, random_state=42, shuffle=True)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

[[1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]]
(749, 1, 1, 2048) (250, 1, 1, 2048) (749, 5) (250, 5)


Train the classifier.

In [46]:
model.fit(X_train, y_train, epochs=20, batch_size=10)

Epoch 1/20
749/749 [==============================] - 2s 2ms/step - loss: 0.5845 - acc: 0.7268
Epoch 2/20
749/749 [==============================] - 1s 2ms/step - loss: 0.5107 - acc: 0.7995
Epoch 3/20
749/749 [==============================] - 1s 2ms/step - loss: 0.5023 - acc: 0.8000
Epoch 4/20
749/749 [==============================] - 1s 2ms/step - loss: 0.5039 - acc: 0.8000
Epoch 5/20
749/749 [==============================] - 1s 2ms/step - loss: 0.5034 - acc: 0.8000
Epoch 6/20
749/749 [==============================] - 1s 2ms/step - loss: 0.5022 - acc: 0.8000
Epoch 7/20
749/749 [==============================] - 1s 2ms/step - loss: 0.5019 - acc: 0.8000
Epoch 8/20
749/749 [==============================] - 1s 2ms/step - loss: 0.4987 - acc: 0.8000
Epoch 9/20
749/749 [==============================] - 1s 2ms/step - loss: 0.5006 - acc: 0.7997
Epoch 10/20
749/749 [==============================] - 1s 2ms/step - loss: 0.4979 - acc: 0.8000
Epoch 11/20
749/749 [============================

## Test the Classifier
Let's test the classifier and see how well it does. Since we only trained on a few images, we are not expecting to win a Kaggle competition, but it will likely get most of the images correct. 

In [47]:
from numpy import argmax

y_probs = model.predict(X_test)
print(y_probs)
print(y_test)
y_prob_max = np.argmax(y_probs, 1)
y_test_max = np.argmax(y_test, 1)
print(y_prob_max)
print(y_test_max)

[[0.28835747 0.16109025 0.18349946 0.16877846 0.24602272]
 [0.29066697 0.16372655 0.17695418 0.17001767 0.25235024]
 [0.2926258  0.16979125 0.17431323 0.16925006 0.2504912 ]
 ...
 [0.28764048 0.16268021 0.17318466 0.17499971 0.2502044 ]
 [0.29237282 0.16624995 0.17816249 0.17222983 0.25339085]
 [0.2910867  0.15479662 0.18316478 0.17671031 0.24936089]]
[[1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 2 0 1 1 0 3 3 4 3 0 1

In [75]:
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
import itertools
import matplotlib
from matplotlib import pyplot as plt

# compute a bunch of classification metrics 
def classification_metrics(y_true, y_pred, y_prob):
    cm_dict = {}
    cm_dict['Accuracy'] = accuracy_score(y_true, y_pred)
    cm_dict['Precision'] =  precision_score(y_true, y_pred)
    cm_dict['Recall'] =  recall_score(y_true, y_pred)
    cm_dict['F1'] =  f1_score(y_true, y_pred) 
    cm_dict['AUC'] = roc_auc_score(y_true, y_prob[:,0])
    cm_dict['Confusion Matrix'] = confusion_matrix(y_true, y_pred).tolist()
    return cm_dict

def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    """Plots a confusion matrix.
    Source: http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    New BSD License - see appendix
    """
    cm_max = cm.max()
    cm_min = cm.min()
    if cm_min > 0: cm_min = 0
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm_max = 1
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    thresh = cm_max / 2.
    plt.clim(cm_min, cm_max)

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i,
                 round(cm[i, j], 3),  # round to 3 decimals if they are float
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    
cm_dict = classification_metrics(y_test_max, y_prob_max, y_probs)
for m in cm_dict:
    print(m, cm_dict[m])
cm = np.asarray(cm_dict['Confusion Matrix'])
plot_confusion_matrix(cm, ['fail','pass'], normalize=False)

ValueError: Target is multiclass but average='binary'. Please choose another average setting.

## Service Definition
Like in the QuickStart notebook our service definition pipeline consists of three stages. Here we use the Keras classifier as the final stage.

In [30]:
from amlrealtimeai.pipeline import ServiceDefinition, TensorflowStage, BrainWaveStage, KerasStage

service_def = ServiceDefinition()
service_def.pipeline.append(TensorflowStage(tf.Session(), in_images, image_tensors))
service_def.pipeline.append(BrainWaveStage(featurizer))
service_def.pipeline.append(KerasStage(model))

service_def_path = os.path.join(datadir, 'save', 'service_def')
service_def.save(service_def_path)
print(service_def_path)

INFO:tensorflow:Froze 0 variables.
Converted 0 variables to const ops.
INFO:tensorflow:Froze 4 variables.
Converted 4 variables to const ops.
/Users/mialiu/Downloads/kagglecatsanddogs_3367a/save/service_def


## Deploy

In [31]:
from amlrealtimeai import DeploymentClient

model_name = "catsanddogs-model"
service_name = "modelbuild-service"

deployment_client = DeploymentClient(subscription_id, resource_group, model_management_account)

The first time the code below runs it will create a new service running your model. If you want to change the model you can make changes above in this notebook and save a new service definition. Then this code will update the running service in place to run the new model.

In [32]:
service = deployment_client.get_service_by_name(service_name)
model_id = deployment_client.register_model(model_name, service_def_path)

Registering model catsanddogs-model
Successfully registered model catsanddogs-model


In [33]:
if(service is None):
    service = deployment_client.create_service(service_name, model_id)    
else:
    service = deployment_client.update_service(service.id, model_id)

Creating service modelbuild-service
. . . . . . . . . 
Successfully created service modelbuild-service


The service is now running in Azure and ready to serve requests. We can check the address and port.

In [34]:
print(service.ipAddress + ':' + str(service.port))

40.121.62.234:80


## Client
There is a simple test client at amlrealtimeai.PredictionClient which can be used for testing. We'll use this client to score an image with our new service.

In [35]:
from amlrealtimeai import PredictionClient
client = PredictionClient(service.ipAddress, service.port)

You can adapt the client [code](../../pythonlib/amlrealtimeai/client.py) to meet your needs. There is also an example C# [client](../../sample-clients/csharp).

The service provides an API that is compatible with TensorFlow Serving. There are instructions to download a sample client [here](https://www.tensorflow.org/serving/setup).

## Request
Let's see how our service does on a few images. It may get a few wrong.

In [36]:
# Specify an image to classify
print('CATS')
for image_file in cat_files[:8]:
    results = client.score_image(image_file)
    result = 'CORRECT ' if results[0] > results[1] else 'WRONG '
    print(result + str(results))
print('DOGS')
for image_file in dog_files[:8]:
    results = client.score_image(image_file)
    result = 'CORRECT ' if results[1] > results[0] else 'WRONG '
    print(result + str(results))

CATS
CORRECT [0.4911853 0.4856984]
CORRECT [0.62519485 0.3544419 ]
CORRECT [0.5897753  0.14872892]
CORRECT [0.5802131  0.35239896]
CORRECT [0.66824925 0.15894377]
CORRECT [0.76457125 0.13966438]
CORRECT [0.8094868 0.3515233]
CORRECT [0.72812706 0.20976284]
DOGS
CORRECT [0.39778802 0.49379167]
CORRECT [0.48429555 0.6952659 ]
CORRECT [0.25493044 0.8189195 ]
CORRECT [0.31072652 0.62333775]
CORRECT [0.2599125 0.6224645]
CORRECT [0.32086092 0.6722173 ]
CORRECT [0.12503223 0.763471  ]
CORRECT [0.40614098 0.40969202]


## Cleanup
Run the cell below to delete your service.

In [37]:
services = deployment_client.list_services()

for service in filter(lambda x: x.name == service_name, services):
    print(service.id)
    deployment_client.delete_service(service.id)
    
models = deployment_client.list_models()

for model in filter(lambda x: x.name == model_name, models):
    print(model.id)
    deployment_client.delete_model(model.id)

4c909171960244e4830b9bce837ba9c3
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
eff4736bda5b4daa907bd025f13c3b53
a9e748ce56184c53a41ee42382432ed0


## Appendix

License for plot_confusion_matrix:

New BSD License

Copyright (c) 2007–2018 The scikit-learn developers.
All rights reserved.


Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

  a. Redistributions of source code must retain the above copyright notice,
     this list of conditions and the following disclaimer.
  b. Redistributions in binary form must reproduce the above copyright
     notice, this list of conditions and the following disclaimer in the
     documentation and/or other materials provided with the distribution.
  c. Neither the name of the Scikit-learn Developers  nor the names of
     its contributors may be used to endorse or promote products
     derived from this software without specific prior written
     permission. 


THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE
ARE DISCLAIMED. IN NO EVENT SHALL THE REGENTS OR CONTRIBUTORS BE LIABLE FOR
ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT
LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY
OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH
DAMAGE.
